___

<a href='https://mainacademy.ua/'> <img src='https://mainacademy.ua/wp-content/uploads/2019/03/logo-main.png' alt = 'Хмм, щось з інтернетом'/></a>
___

# Module 4: Work with data professionally!

## Lab work 4



#### Мета: 

* навчитися обробляти дані
* "парсити" сайти

### Завдання 1:

#### Ознайомитися із API Національного банку 
https://old.bank.gov.ua/control/uk/publish/article?art_id=38441973&cat_id=38459171#exchange



1. Підключитися до API.


In [1]:
import requests
import json
import pickle
from bs4 import BeautifulSoup
from datetime import timedelta, datetime

enddate = datetime.now()
startdate = enddate - timedelta(days=365)
enddate_str = enddate.strftime("%Y%m%d")
startdate_str = startdate.strftime("%Y%m%d")
min_gov_link = requests.get(f"https://bank.gov.ua/NBU_Exchange/exchange_site?start={startdate_str}&end={enddate_str}&valcode=usd&sort=exchangedate&order=desc&json")


2. Отримати курс долара за останній рік.

In [2]:
min_gov_dict = json.loads(min_gov_link.text)

3. Вивести середнє значення та відхилення курсу за кожний місяць.

In [3]:
#Отримуємо середнє значення для кожного місяця
my_dict = {}
for st in min_gov_dict:
    if st['exchangedate'][3:] in my_dict:
        a += 1
        my_dict[st['exchangedate'][3:]] = {'average' : round((my_dict[st['exchangedate'][3:]]['average'] * (a - 1) + float(st['rate'])) / a, 2)}
    else:
        a = 1
        my_dict[st['exchangedate'][3:]] = {'average': round(float(st['rate']), 2)}

#Отримуємо відхилення середнього значення для кожного місяця
a = my_dict[min_gov_dict[0]['exchangedate'][3:]]['average']
b = []
for key, value in my_dict.items():
    #value['deviation'] = round(a - value['average'], 2)
    b.append(round(a - value['average'], 2))
    a = value['average']
a = 0
for key, value in my_dict.items():
    if a + 1 < len(b):
        a += 1
        value['deviation'] = b[a]
    else:
        value['deviation'] = 0.0

4. Дану інформацію записати у файл за допомогою pickle.

In [4]:
with open('DS_Practice4_task1.pickle', 'wb') as f:
    pickle.dump(my_dict, f)

### Завдання 2:

Потрібно проаналізувати всі товари на сайті: 
https://smallpacking.agrosem.ua/shop/

Створіть базу даних, що містить таблицю із такими стовбцями: 
    - артикул товару
    - найменування
    - ціна 
    - вага

In [177]:
import requests
from bs4 import BeautifulSoup
import sqlite3
# Підключаємося до сайту
page = requests.get('https://smallpacking.agrosem.ua/shop/')
soup = BeautifulSoup(page.text, 'html.parser')
prod_div = soup.find('div', 'product-category-wrapper').find_all('div', 'product-from-category-container')
page_list = range(1, int((soup.find('ul', 'page-numbers').find_all('li')[-2]).text)+1) # Отримуємо к-кість сторінок
my_base = {}
# Проходимо по кожній сторінці
for j in page_list:
    page = requests.get(f'https://smallpacking.agrosem.ua/shop/page/{j}/')
    soup = BeautifulSoup(page.text, 'html.parser')
    prod_div = soup.find('div', 'product-category-wrapper').find_all('div', 'product-from-category-container')
    # Проходимо по кожній карточці
    for obj in prod_div:
        prod_link = requests.get(obj.find('a').get('href'))
        prod_link_soup = BeautifulSoup(prod_link.text, "html.parser")
        art = prod_link_soup.find('span', itemprop="productID").text
        # Перевіряємо наявність параметру "Вага"
        k = 'woocommerce-product-attributes-item woocommerce-product-attributes-item--attribute_pa_vaga'
        if k not in prod_link_soup.prettify():
            k = ""
        else:
            k = prod_link_soup.find('tr', 'woocommerce-product-attributes-item woocommerce-product-attributes-item--attribute_pa_vaga').find('td', 'woocommerce-product-attributes-item__value').text
        prc = str(prod_link_soup.find('span', 'regular-price').text[:-3].replace(' ', '')).replace(',', '.')
        wght = str(k[:-3].replace(' ', '')).replace(',', '.')
        if len(prc) > 0:
            prc = float(prc)
        else:
            prc = "NULL"
        if len(wght) > 0:
            wght = float(wght)
        else:
            wght = "NULL"
        my_base[art] = {
            'Name' : prod_link_soup.find('div', 'atributes-description').h2.text,
            'Price' : prc,
            'Weight' : wght
        }
# Cтворюємо базу даних
conn = sqlite3.connect('my_db.db')
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS PRODUCTS")
sql ='''CREATE TABLE PRODUCTS(
   SKU PRIMARY KEY,
   PROD_NAME TEXT,
   PRICE FLOAT(2),
   WEIGHT FLOAT(2)
)'''
# Записуємо у дб дані
cursor.execute(sql)
for key, value in my_base.items():
    sc = "44"
    query = f'''INSERT INTO PRODUCTS(SKU, PROD_NAME, PRICE, WEIGHT)
    VALUES({key[5:]}, "{(value["Name"])}", "{value["Price"]}", "{value["Weight"]}")'''
    cursor.execute(query)
conn.commit()
conn.close()

{'SKU: 42522': {'Name': '8Х288КЛДМ (високоолеїновий)', 'Price': 'NULL', 'Weight': 'NULL'}, 'SKU: 66336': {'Name': '8Х477КЛ (високоолеїновий)', 'Price': 'NULL', 'Weight': 'NULL'}, 'SKU: 91797': {'Name': 'AMELIOR (ФАО 240)', 'Price': 'NULL', 'Weight': 'NULL'}, 'SKU: 28164': {'Name': 'Ferticare Hydro 6-14-30 + micro', 'Price': 2500.0, 'Weight': 25.0}, 'SKU: 66353': {'Name': 'Ferticare Kombi 14-11-25 + micro', 'Price': 2250.0, 'Weight': 25.0}, 'SKU: 39566': {'Name': 'MagPlon K (нітрат калію)', 'Price': 930.0, 'Weight': 25.0}, 'SKU: 94424': {'Name': 'MAS 10.A (ФАО 180)', 'Price': 'NULL', 'Weight': 'NULL'}, 'SKU: 94116': {'Name': 'MAS 15.T (ФАО 220)', 'Price': 'NULL', 'Weight': 'NULL'}, 'SKU: 63652': {'Name': 'MAS 18.L (ФАО 210)', 'Price': 'NULL', 'Weight': 'NULL'}, 'SKU: 79663': {'Name': 'MAS 20.A (ФАО 230) – Ексклюзивний гібрид ТОВ «АГРОСЕМ»', 'Price': 'NULL', 'Weight': 'NULL'}, 'SKU: 88745': {'Name': 'MAS 20.F (ФАО 230)', 'Price': 'NULL', 'Weight': 'NULL'}, 'SKU: 65361': {'Name': 'MAS 23.

### Завдання 3: 

Ознайомтеся із роботою SQLite та відповідним модулем у Python.
Завантажте базу даних для виконання лабораторної роботи.
Підключіться до завантаженої бази SQLite.

1. Виведіть інформацію про дану базу.

In [1]:
import sqlite3
conn = sqlite3.connect('tysql.sqlite')
cursor = conn.cursor()
cursor.execute("SELECT sql FROM sqlite_master WHERE type='table';")
result = cursor.fetchall()
for i in result:
    print(i[0])

CREATE TABLE Customers
(
  cust_id      char(10)  NOT NULL ,
  cust_name    char(50)  NOT NULL ,
  cust_address char(50)  NULL ,
  cust_city    char(50)  NULL ,
  cust_state   char(5)   NULL ,
  cust_zip     char(10)  NULL ,
  cust_country char(50)  NULL ,
  cust_contact char(50)  NULL ,
  cust_email   char(255) NULL ,
  PRIMARY KEY (cust_id)
)
CREATE TABLE OrderItems
(
  order_num  int          NOT NULL                      ,
  order_item int          NOT NULL                      ,
  prod_id    char(10)     NOT NULL                      ,
  quantity   int          NOT NULL                      ,
  item_price decimal(8,2) NOT NULL                      ,
  PRIMARY KEY (order_num, order_item)                   ,
  FOREIGN KEY (order_num) REFERENCES Orders (order_num) ,
  FOREIGN KEY (prod_id) REFERENCES Products (prod_id)
)
CREATE TABLE Orders
(
  order_num  int      NOT NULL ,
  order_date datetime NOT NULL ,
  cust_id    char(10) NOT NULL ,
  PRIMARY KEY (order_num)      ,
  FOREIGN K

2. Виведіть перелік всіх таблиць.

In [2]:
for i in result:
    i = str(i[0])
    print(i.split(sep = "(")[0][13:])

Customers

OrderItems

Orders

Products

Vendors



3. Список всіх cust_id з таблиці Customers table.

In [3]:
cursor.execute("SELECT cust_id FROM Customers")
print(cursor.fetchall())

[('1000000001',), ('1000000002',), ('1000000003',), ('1000000004',), ('1000000005',)]


4. Всю таблицю Customers table

In [4]:
cursor.execute("SELECT * FROM Customers")
print(cursor.fetchall())

[('1000000001', 'Village Toys', '200 Maple Lane', 'Detroit', 'MI', '44444', 'USA', 'John Smith', 'sales@villagetoys.com'), ('1000000002', 'Kids Place', '333 South Lake Drive', 'Columbus', 'OH', '43333', 'USA', 'Michelle Green', None), ('1000000003', 'Fun4All', '1 Sunny Place', 'Muncie', 'IN', '42222', 'USA', 'Jim Jones', 'jjones@fun4all.com'), ('1000000004', 'Fun4All', '829 Riverside Drive', 'Phoenix', 'AZ', '88888', 'USA', 'Denise L. Stephens', 'dstephens@fun4all.com'), ('1000000005', 'The Toy Store', '4545 53rd Street', 'Chicago', 'IL', '54545', 'USA', 'Kim Howard', None)]


5. Список клієнтів (cust_names) відсортованих від Z до A

In [74]:
cursor.execute("SELECT cust_name FROM Customers ORDER BY cust_name DESC")
print(cursor.fetchall())

[('Village Toys',), ('The Toy Store',), ('Kids Place',), ('Fun4All',), ('Fun4All',)]


6. Таблицю клієнтів та замовлень (cust_id і order_num) . Відсортуйте по клієнту і потім по даті замовлення

In [9]:
cursor.execute("SELECT cust_id, order_num FROM Orders ORDER BY cust_id, order_num")
print(cursor.fetchall())

[('1000000001', 20005), ('1000000001', 20009), ('1000000003', 20006), ('1000000004', 20007), ('1000000005', 20008)]


7. Таблицю (на основі Items) з кількість та вартістю товару. Відсортуйте в порядку спадання по кількості та вартості

In [12]:
cursor.execute("""
SELECT prod_id, quantity, item_price FROM OrderItems ORDER BY quantity DESC, item_price DESC""")
print(cursor.fetchall())

[('BNBG01', 250, 2.49), ('BNBG02', 250, 2.49), ('BNBG03', 250, 2.49), ('BR03', 100, 10.99), ('BR01', 100, 5.49), ('BNBG01', 100, 2.99), ('BNBG02', 100, 2.99), ('BNBG03', 100, 2.99), ('BR03', 50, 11.49), ('RGAN01', 50, 4.49), ('BR01', 20, 5.99), ('BR03', 10, 11.99), ('BR02', 10, 8.99), ('BNBG01', 10, 3.49), ('BNBG02', 10, 3.49), ('BNBG03', 10, 3.49), ('BR03', 5, 11.99), ('RGAN01', 5, 4.99)]


8. Товар (з таблиці Products), ціна якого становить 9.49

In [13]:
cursor.execute("SELECT prod_name FROM Products WHERE prod_price IS 9.49")
print(cursor.fetchall())

[('King doll',), ('Queen doll',)]


9. Виведіть назву товару та ціну, яка лежить в діапазоні від  3 до 6. Відсортуйте результат в по ціні в порядку зростання

In [14]:
cursor.execute("SELECT prod_name, prod_price FROM Products WHERE prod_price BETWEEN 3 AND 6 ORDER BY prod_price")
print(cursor.fetchall())

[('Fish bean bag toy', 3.49), ('Bird bean bag toy', 3.49), ('Rabbit bean bag toy', 3.49), ('Raggedy Ann', 4.99), ('8 inch teddy bear', 5.99)]


10. Кількість товару, що було продано

In [17]:
cursor.execute("SELECT SUM(quantity) FROM OrderItems ")
print(cursor.fetchall())

[(1430,)]


11. Кількість найменувань товару, ціна якого більша за 4

In [18]:
cursor.execute("SELECT COUNT(prod_name) FROM Products WHERE prod_price > 4")
print(cursor.fetchall())

[(6,)]


12. Розробіть алгоритм для виведення 3 найдорожчих товарів в базі

In [25]:
cursor.execute("SELECT prod_name FROM Products GROUP BY prod_name ORDER BY MAX(prod_price) DESC LIMIT 3")
print(cursor.fetchall())

[('18 inch teddy bear',), ('Queen doll',), ('King doll',)]


13. Підрахуйте кількість замовлень для кожного клієнта використовуючи підзапити

In [28]:
cursor.execute("SELECT cust_name, COUNT(order_num) FROM Orders INNER JOIN Customers ON Orders.cust_id = Customers.cust_id GROUP BY cust_name")
print(cursor.fetchall())

[('Fun4All', 2), ('The Toy Store', 1), ('Village Toys', 2)]


14. Виведіть список клієнтів та їх замовлення (використайте JOIN)

In [33]:
cursor.execute("""SELECT cust_name, prod_name
                FROM Customers
                LEFT JOIN Orders ON Orders.cust_id = Customers.cust_id
                LEFT JOIN OrderItems ON Orders.order_num = OrderItems.order_num
                LEFT JOIN Products ON OrderItems.prod_id = Products.prod_id
                GROUP BY cust_name""")
print(cursor.fetchall())

[('Fun4All', '8 inch teddy bear'), ('Kids Place', None), ('The Toy Store', 'Raggedy Ann'), ('Village Toys', '8 inch teddy bear')]


15. Зробіть два запити: список товарів, ціна яких менша 5 та список товарів, ціна який більша рівна 5. Використайте UNION для об'єднання цих запитів

In [35]:
cursor.execute("SELECT prod_name FROM Products WHERE prod_price < 5 UNION SELECT prod_name FROM Products WHERE prod_price >= 5")
print(cursor.fetchall())

[('12 inch teddy bear',), ('18 inch teddy bear',), ('8 inch teddy bear',), ('Bird bean bag toy',), ('Fish bean bag toy',), ('King doll',), ('Queen doll',), ('Rabbit bean bag toy',), ('Raggedy Ann',)]


### Завдання 4:

Зареєструйтеся на сайті  http://www.sql-ex.ru/

Виконайте завдання із блоку SELECT; з 1 по 30 завдання

Код запитів вставляйте сюди


In [ ]:
#Завдання 1
SELECT Product.model, PC.speed, PC.hd
FROM Product
LEFT JOIN PC on Product.model = PC.model
WHERE PC.price < 500
#Завдання 2
SELECT DISTINCT maker
FROM Product
WHERE type = 'Printer'
#Завдання 3
SELECT model, ram, screen
FROM Laptop
WHERE price > 1000
#Завдання 4
SELECT code, model, color, type, price
FROM printer
WHERE color='y'
#Завдання 5
SELECT model, speed, hd
FROM PC
WHERE price > 600 AND (cd="12x" OR cd="24x")
#Завдання 6
SELECT DISTINCT Product.maker, Laptop.speed
FROM Product, Laptop
WHERE Product.model = Laptop.model
AND Laptop.hd >= 10
#Завдання 7
SELECT PC.model, price
FROM PC INNER JOIN Product ON PC.model = Product.model
WHERE maker = 'B'
GROUP BY PC.model, price
UNION
SELECT Laptop.model, price
FROM Laptop INNER JOIN Product ON Laptop.model = Product.model
WHERE maker = 'B'
GROUP BY Laptop.model, price
UNION
SELECT Printer.model, price
FROM Printer INNER JOIN Product ON Printer.model = Product.model
WHERE maker = 'B'
GROUP BY Printer.model, price
#Завдання 8
SELECT maker
FROM Product
WHERE maker NOT IN (SELECT maker
FROM Product
WHERE model IN (SELECT model FROM Laptop)) AND model IN (SELECT model FROM PC)
GROUP BY maker
#Завдання 9
SELECT maker
FROM Product INNER JOIN PC ON Product.model = PC.model
WHERE speed >= 450
GROUP BY maker
#Завдання 10
SELECT model, price
FROM Printer
WHERE price = (SELECT MAX(price) FROM printer)
#Завдання 11
SELECT AVG(speed)
FROM PC
#Завдання 12
SELECT AVG(speed)
FROM Laptop
WHERE price > 1000
#Завдання 13
SELECT AVG(speed)
FROM PC INNER JOIN Product ON PC.model = Product.model
WHERE maker = 'A'
#Завдання 14
SELECT Ships.class, name, country
FROM Ships INNER JOIN Classes ON Ships.class = Classes.class
WHERE numGuns >= 10
#Завдання 15
SELECT hd
FROM PC
GROUP BY hd
HAVING COUNT(hd) > 1
#Завдання 16
SELECT MAX(model), MIN(model), speed, RAM
FROM PC
GROUP BY speed, RAM
HAVING COUNT(speed) > 1 AND MAX(model) <> MIN(model)
#Завдання 17
SELECT type, Laptop.model, speed
FROM Laptop INNER JOIN Product ON Laptop.model = Product.model
WHERE speed < ALL (SELECT speed
FROM PC)
#Завдання 18
SELECT maker, price
FROM Product INNER JOIN Printer ON Product.model = Printer.model
WHERE price = (SELECT MIN(price) FROM Printer WHERE color = 'y') AND color = 'y'

#Завдання 19
SELECT maker, AVG(screen)
FROM Product INNER JOIN Laptop ON Product.model = Laptop.model
GROUP BY maker

#Завдання 20
SELECT Maker, COUNT(model) AS Count_Model
FROM Product
WHERE type = 'PC'
GROUP BY Maker
HAVING COUNT(model) > 2
#Завдання 21
SELECT maker, MAX(price) AS Max_price
FROM Product INNER JOIN PC ON Product.model = PC.model
GROUP BY maker

#Завдання 22
SELECT speed, AVG(price)
FROM PC
WHERE speed > 600
GROUP BY speed
#Завдання 23
SELECT maker
FROM Product
WHERE model IN (SELECT model FROM PC WHERE speed >= 750
INTERSECT
SELECT model FROM PC WHERE speed >= 750)
GROUP BY maker
#Завдання 24
SELECT model
FROM (SELECT model 'model', price
FROM PC
UNION ALL
SELECT model 'model', price
FROM Laptop
UNION ALL
SELECT model 'model', price
FROM Printer) X
WHERE price >= ALL(SELECT price
FROM PC
UNION ALL
SELECT price
FROM Laptop
UNION ALL
SELECT price
FROM Printer)
#Завдання 25
SELECT maker
FROM Product
WHERE model IN
((SELECT model FROM PC WHERE ram = (SELECT MIN(ram) FROM PC))
UNION ALL
(SELECT model FROM PC WHERE speed = (SELECT MAX(speed) FROM PC)))
GROUP BY maker
#Завдання 26
SELECT AVG(price)
FROM (SELECT price 'Price'
FROM PC INNER JOIN Product ON PC.model = Product.model
WHERE maker = 'A'
UNION
SELECT price 'Price'
FROM Laptop INNER JOIN Product ON Laptop.model = Product.model
WHERE maker = 'A') X

#Завдання 27
SELECT maker, (SELECT AVG(hd) FROM PC)
FROM printer INNER JOIN Product ON printer.model = Product.model
WHERE maker IN (SELECT maker FROM PC INNER JOIN Product ON PC.model = Product.model)
GROUP BY maker
#Завдання 28
SELECT COUNT(maker) AS qty
FROM
(SELECT maker
FROM (SELECT maker, PC.model
FROM PC INNER JOIN Product ON PC.model = Product.model
UNION
SELECT maker, Laptop.model
FROM Laptop INNER JOIN Product ON Laptop.model = Product.model
UNION
SELECT maker, Printer.model
FROM Printer INNER JOIN Product ON Printer.model = Product.model) X
GROUP BY maker
HAVING COUNT(maker) = 1) Y
#Завдання 29
SELECT Income.point, Income.date, inc, out
FROM Income LEFT JOIN Outcome ON Income.code = Outcome. code
#Завдання 30
SELECT Income.point, Income.date, SUM(inc), SUM(out)
FROM Income INNER JOIN Outcome ON Income.point = Outcome.point
GROUP BY Income.point, Income.date
